In [1]:
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
import matplotlib.axes as axe
import pandas as pd
import datetime as dt
import gurobipy as gp
from gurobipy import GRB
import cvxpy as cp

import random
from itertools import chain, combinations, tee
import time

plt.rcParams['text.usetex'] = True

# Functions

In [2]:
def demand_name_by_group_index(index):
    list_demand_names = ["Demand (eligible group, 1)", "Demand (eligible group, 2)", \
                         "Demand (ineligible group, 1)", "Demand (ineligible group, 2)", \
                         "Demand (ineligible group, 3)"]
    return list_demand_names[index]

def VoT_name_by_group_index(index):
    list_demand_names = ["VoT (eligible group, 1)", "VoT (eligible group, 2)", \
                         "VoT (ineligible group, 1)", "VoT (ineligible group, 2)", \
                         "VoT (ineligible group, 3)"]
    return list_demand_names[index]


# Groups, Routes to Edges:

In [3]:
directory_path = '../data/data_income_percentage_VoT___101_N_Sep_to_Nov_2024/'
df_data = pd.read_csv(directory_path + 'data_cities_od_VoTs_demands_1.csv')

# df_od_flow_data
# df_data

In [4]:
dict_data = {}

for column_name_full in list(df_data.columns):
    if column_name_full == "Data Category":
        categories_list = df_data[column_name_full].tolist()
    else:
        dict_data[int(column_name_full)] = {}
        for category_index, category in enumerate(categories_list):
            if category == "Start City Index" or category == "End City Index":
                dict_data[int(column_name_full)][category] \
                    = int(df_data[column_name_full].tolist()[category_index])
            elif category == "Start City" or category == "End City":
                dict_data[int(column_name_full)][category] \
                    = df_data[column_name_full].tolist()[category_index]
            else:
#                 print("category:", category)
                dict_data[int(column_name_full)][category] \
                    = float(df_data[column_name_full].tolist()[category_index])


In [5]:
dict_data

{0: {'Start City Index': 0,
  'End City Index': 0,
  'Start City': 'Palo Alto',
  'End City': 'Palo Alto',
  'O-D Flow (Max Entropy)': 612.539616936298,
  'Demand (eligible group, 1)': 47.77809012103125,
  'VoT (eligible group, 1)': 0.03197423570019724,
  'Demand (eligible group, 2)': 27.564282762133406,
  'VoT (eligible group, 2)': 0.10238603988603988,
  'Demand (ineligible group, 1)': 115.15744798402402,
  'VoT (ineligible group, 1)': 0.2754407051282051,
  'Demand (ineligible group, 2)': 134.75871572598555,
  'VoT (ineligible group, 2)': 0.5809294871794872,
  'Demand (ineligible group, 3)': 287.28108034312373,
  'VoT (ineligible group, 3)': 1.8596449415012848},
 1: {'Start City Index': 0,
  'End City Index': 1,
  'Start City': 'Palo Alto',
  'End City': 'East Palo Alto',
  'O-D Flow (Max Entropy)': 98.03491986336807,
  'Demand (eligible group, 1)': 7.64672374934271,
  'VoT (eligible group, 1)': 0.03197423570019724,
  'Demand (eligible group, 2)': 4.411571393851563,
  'VoT (eligible g

In [6]:
cities_dict = {}
for od_info in list(dict_data.values()):
    if od_info["Start City Index"] not in list(cities_dict.keys()):
        cities_dict[od_info["Start City Index"]] = od_info["Start City"]
    if od_info["End City Index"] not in list(cities_dict.keys()):
        cities_dict[od_info["End City Index"]] = od_info["End City"]

cities_list = list(cities_dict.values())

# cities_dict

In [7]:
od_to_edges_array = np.zeros((len(list(dict_data.keys())), 2))

for od_index, od_info in dict_data.items():
    od_to_edges_array[od_index, 0] = int(cities_list.index(od_info["Start City"]))
    od_to_edges_array[od_index, 1] = int(cities_list.index(od_info["End City"]))

# od_to_edges_array

In [8]:
num_groups_per_od = 5

demand_array = np.zeros((len(list(dict_data.keys())), num_groups_per_od))
VoT_array_base = np.zeros((len(list(dict_data.keys())), num_groups_per_od))

for od_index, od_value in dict_data.items():
    for group_index in range(num_groups_per_od):
        demand_name = demand_name_by_group_index(group_index)
        VoT_name = VoT_name_by_group_index(group_index)
        
        demand_array[od_index, group_index] = od_value[demand_name]
        VoT_array_base[od_index, group_index] = od_value[VoT_name]

# print(demand_array)
# VoT_array_base

In [9]:
directory_path = '../data/data_income_percentage_VoT___101_N_Sep_to_Nov_2024/'

VoT_array = np.zeros((VoT_array_base.shape[0], VoT_array_base.shape[1], 5))

T = 5
for t in range(T):
    df_perturbation_data = pd.read_csv(directory_path + 'perturbations_1_' + str(t) + '.csv')
    perturbation_array = df_perturbation_data.to_numpy()[:, 1:]
    VoT_array[:, :, t] = VoT_array_base * perturbation_array
    
# VoT_array_base
# perturbation_array

# Download Latency Parameters Data

In [10]:
directory_path_latency = '../data/pems_latency_inference___101_N_Sep_to_Nov_2024/'
df_latency_params = pd.read_csv(directory_path_latency + 'latency_params.csv')

# list(df_latency_params.loc[:, "Palo Alto"])

In [11]:
dict_latency_params = {}

city_list = list(df_latency_params.columns)[1:]

for city in city_list:
#     if city != "Belmont":
    if 1 == 1:
        dict_latency_params[city] = {}
        dict_latency_params[city]["Flow (at bend)"] = df_latency_params.loc[:, city][0]
        dict_latency_params[city]["Latency (at bend)"] = df_latency_params.loc[:, city][1]
        dict_latency_params[city]["Slope (after bend)"] = df_latency_params.loc[:, city][2]

dict_latency_params

{'Palo Alto': {'Flow (at bend)': 861.9885,
  'Latency (at bend)': 1.326448252,
  'Slope (after bend)': 0.000782666},
 'East Palo Alto': {'Flow (at bend)': 1001.517857,
  'Latency (at bend)': 2.213126553,
  'Slope (after bend)': 0.000584484},
 'Redwood City': {'Flow (at bend)': 881.1846667,
  'Latency (at bend)': 4.892192375,
  'Slope (after bend)': 0.001563724},
 'Belmont': {'Flow (at bend)': 1278.948125,
  'Latency (at bend)': 1.199911179,
  'Slope (after bend)': 0.001994138},
 'San Mateo': {'Flow (at bend)': 1034.092826,
  'Latency (at bend)': 5.541006284,
  'Slope (after bend)': 0.002147262},
 'Burlingame': {'Flow (at bend)': 845.15,
  'Latency (at bend)': 1.503111345,
  'Slope (after bend)': 0.000306601},
 'Millbrae': {'Flow (at bend)': 853.1818182,
  'Latency (at bend)': 2.384328452,
  'Slope (after bend)': 0.000321856}}

# General CBCP Equilibrium Solver

## (Special Case) Quartic Polynomial Latency Functions

In [12]:
# grad = np.array([0, 1, 2, 3, 4])
grad = np.array([2, 4, 0, 1, 3])

for id_temp, entry_temp in enumerate(grad):
    print("id_temp, entry_temp:", id_temp, entry_temp)


id_temp, entry_temp: 0 2
id_temp, entry_temp: 1 4
id_temp, entry_temp: 2 0
id_temp, entry_temp: 3 1
id_temp, entry_temp: 4 3


In [13]:
# np.array([1, 2, 3]) * np.array([6, 5, 3])

In [14]:
def coeff_from_input(coeff_input, num_edges, num_gp_lanes):
    # Goal: Differentiate between express and general purpose lanes.
    
    assert np.all(coeff_input >= 0.0), "All entries of coeff_input must be non-negative."
    assert len(coeff_input.shape) == 2, "coeff_input should be a matrix, i.e., 2-D array"
    assert coeff_input.shape[0] == 3, "Latency functions are assumed to be piecewise linear / affine with 3 parameters."
    assert coeff_input.shape[1] == num_edges, "Latency functions should be defined across all edges."
    
    coeff_length = 3
    
    ex_to_gp_multiplier = np.array([1, 1/num_gp_lanes, num_gp_lanes]).reshape((coeff_length, 1)) \
                            @ np.ones((1, num_edges))
    
    coeff_full = np.zeros((coeff_length, num_edges, 2))
    coeff_full[:, :, 0] = coeff_input
    coeff_full[:, :, 1] = coeff_input * ex_to_gp_multiplier
    
    return coeff_full

def latency_max(flow_max, coeff):
    
    assert np.all(coeff >= 0.0), "coeff should be non-negative"
    assert len(coeff.shape) == 1, "coeff should be a 1-D array."
    assert coeff.shape[0] == 3, "Latency functions are assumed to be piecewise linear / affine with 3 parameters."
    
    return coeff[0] + max(coeff[1] * (flow_max - coeff[2]), 0)

def welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, tau, \
                demand_array, VoT_array, num_el, od_to_edges_array, y, \
                coeff_input):

    coeff = coeff_from_input(coeff_input, num_edges, num_gp_lanes)
    
    assert len(od_to_edges_array.shape) == 2, "od_to_edges should be 2-dimensional."
    assert od_to_edges_array.shape[1] == 2, "od_to_edges second dimension should be for start and end edges."
    
    edge_to_od_dict = {}
    for e in range(num_edges):
        edge_to_od_dict[e] = [k for k in list(range(int(od_to_edges_array.shape[0]) )) \
                               if od_to_edges_array[k, 0] <= e <= od_to_edges_array[k, 1]]

    num_groups = demand_array.shape[1]
    num_in = num_groups - num_el
    assert num_in >= 0, "We must have num_in >= 0."
    
    el_indices = list(range(num_el))
    in_indices = list(range(num_el, num_groups))
    
#     print()
#     print("tau.shape[0]:", tau.shape[0])
#     print("num_edges:", num_edges)
#     print()

    assert len(tau.shape) == 2, "tau should be 2-dimensional."
    assert tau.shape[0] == num_edges, "toll vector's first axis length must equal the number of edges."
    assert tau.shape[1] == T, "toll vector's second axis length must equal the time horizon."
    
    ## Compute lane flows:
    
    x = np.zeros((num_edges, 2, T))
    for e in range(num_edges):
        for t in range(T):
            x[e, 0, t] += sum(y[(od, g, e, 0, t)] for od in edge_to_od_dict[e] \
                              for g in range(num_groups))
    for e in range(num_edges):
        for t in range(T):
            x[e, 1, t] += sum( (y[(od, g, e, 0, t)] + y[(od, g, e, 1, t)]) for od in edge_to_od_dict[e] \
                              for g in el_indices)
            x[e, 1, t] += sum(y[(od, g, e, 1, t)] for od in edge_to_od_dict[e] \
                              for g in in_indices)
            
    print()
    print("in_indices:", in_indices)
    print("el_indices:", el_indices)
    print()
    
    ## Compute lane latencies:
    
    ell = np.zeros((num_edges, 2, T))
    for e in range(num_edges):
        for t in range(T):
            ell[e, 0, t] = coeff[0, e, 0] + coeff[1, e, 0] * max(x[e, 0, t] - coeff[2, e, 0], 0)
            ell[e, 1, t] = coeff[0, e, 1] + coeff[1, e, 1] * max(x[e, 1, t]/num_gp_lanes - coeff[2, e, 1], 0)

    obj_E = sum( y[(od, g, e, 0, t)] * VoT_array[od, g, t] * ell[e, 0, t] \
                for e in range(num_edges) for od in edge_to_od_dict[e] \
                for g in el_indices for t in range(T) ) \
            + sum( y[(od, g, e, 1, t)] * (VoT_array[od, g, t] * ell[e, 0, t] + tau[e, t]) \
                for e in range(num_edges) for od in edge_to_od_dict[e] \
                for g in el_indices for t in range(T) ) \
            + sum( y[(od, g, e, 2, t)] * VoT_array[od, g, t] * ell[e, 1, t] \
                  for e in range(num_edges) for od in edge_to_od_dict[e] \
                  for g in el_indices for t in range(T) )
    obj_I = sum( y[(od, g, e, 0, t)] * (VoT_array[od, g, t] * ell[e, 0, t] + tau[e, t]) \
                for e in range(num_edges) for od in edge_to_od_dict[e] \
                for g in in_indices for t in range(T) ) \
            + sum( y[(od, g, e, 1, t)] * VoT_array[od, g, t] * ell[e, 1, t] \
                for e in range(num_edges) for od in edge_to_od_dict[e] \
                  for g in in_indices for t in range(T) )
    obj_R = sum( y[(od, g, e, 0, t)] * tau[e, t] \
                for e in range(num_edges) for od in edge_to_od_dict[e] \
                for g in in_indices for t in range(T) )

    welfare = lambda_E * obj_E - lambda_R * obj_R + lambda_I * obj_I

#     print()
#     print("obj_E:", obj_E)
#     print("obj_R:", obj_R)
#     print("obj_I:", obj_I)
#     print("welfare:", welfare)
#     print()
    
    return welfare, obj_E, obj_R, obj_I


# Latency, total, throughout the entire time horizon. 

def latency_total(T, num_edges, lambda_E, lambda_R, lambda_I, tau, \
                  demand_array, VoT_array, num_el, y, od_to_edges_array, \
                  coeff_input):
    
    coeff = coeff_from_input(coeff_input, num_edges, num_gp_lanes)
    
    assert len(od_to_edges_array.shape) == 2, "od_to_edges should be 2-dimensional."
    assert od_to_edges_array.shape[1] == 2, "od_to_edges' second dimension should be for start and end edges."
    
    edge_to_od_dict = {}
    for edge in range(num_edges):
        edge_to_od_dict[e] = [k for k in list(range(int(od_to_edges_array.shape[0]) )) \
                               if od_to_edges_array[k, 0] <= e <= od_to_edges_array[k, 1]]
    
    # In full:
    # y indices: (od, group, edge, "lane", time)

    num_groups = demand_array.shape[1]
    num_in = num_groups - num_el
    assert num_in >= 0, "We must have num_in >= 0."
    
    el_indices = list(range(num_el))
    in_indices = list(range(num_el, num_groups))
    
#     print()
#     print("tau.shape[0]:", tau.shape[0])
#     print("num_edges:", num_edges)
#     print()

    assert len(tau.shape) == 2, "tau should be 2-dimensional."
    assert tau.shape[0] == num_edges, "toll vector's first axis length must equal the number of edges."
    assert tau.shape[1] == T, "toll vector's second axis length must equal the time horizon."
    
    ## Compute lane flows:
    
    x = np.zeros((num_edges, 2, T))
    for e in range(num_edges):
        for t in range(T):
            x[e, 0, t] += sum(y[(od, g, e, 0, t)] for od in edge_to_od_dict[e] \
                              for g in range(num_groups))
    for e in range(num_edges):
        for t in range(T):
            x[e, 1, t] += sum( (y[(od, g, e, 0, t)] + y[(od, g, e, 1, t)]) for od in edge_to_od_dict[e] \
                              for g in el_indices)
            x[e, 1, t] += sum(y[(od, g, e, 1, t)] for od in edge_to_od_dict[e] \
                              for g in in_indices)
    
    ## Compute lane latencies:
    
    ell = np.zeros((num_edges, 2, T))
    for e in range(num_edges):
        for t in range(T):
            ell[e, 0, t] = coeff[0, e, 0] + coeff[1, e, 0] * max(x[e, 0, t] - coeff[2, e, 0], 0)
            ell[e, 1, t] = coeff[0, e, 1] + coeff[1, e, 1] * max(x[e, 1, t] - coeff[2, e, 1], 0)

    latency = sum( x[e, k, t] * ell[e, k, t] \
                for e in range(num_edges) for k in range(2) for t in range(T) )

#     print()
#     print("obj_E:", obj_E)
#     print("obj_R:", obj_R)
#     print("obj_I:", obj_I)
#     print("welfare:", welfare)
#     print()
    
    return latency

In [15]:
# arr = np.arange(5)
# arr.reshape((5, 1))

In [32]:
def proj_tau_B(T, num_edges, tau, B, od_to_edges_list_full, tau_max, B_max = 1.0, B_min = 0.1):

#     print()
#     print("tau.shape[0]:", tau.shape[0])
#     print("num_edges:", num_edges)
#     print()

    assert tau.shape[0] == num_edges, "tau must have length equal to the number of edges."
    assert tau_max.shape[0] == num_edges, "tau_max must have length equal to the number of edges."
    
    tau_feas = cp.Variable((num_edges, T))
#     B_feas = cp.Variable(1)
    
    func = cp.sum_squares(tau_feas - tau)

    objective = cp.Minimize(func)

    constraints = []
    constraints += [tau_feas >= 0.0]
#     constraints += [B_feas >= 0.0]
    constraints += [tau_feas <= tau_max]
#     constraints += [B_feas <= B_max]

#     constraints += [B_feas <= sum(tau_feas[e, t] for e in od_to_edges_list_full[od] for t in range(T)) \
#                     for od in range(len(od_to_edges_list_full))]
    
    prob = cp.Problem(objective, constraints)
    result = prob.solve()

#     print()
#     print("tau_feas.value:", np.round(np.maximum(tau_feas.value, 0.0), decimals=4))
#     print()

    B_max_list = [sum(tau_feas.value[e, t] for e in od_to_edges_list_full[od] for t in range(T)) \
                            for od in range(len(od_to_edges_list_full))]
#     B_max_wrt_tau_feas = min([B_max for B_max in B_max_list if B_max >= B_min])
    B_max_wrt_tau_feas = max(B_max_list)
    B_feas = min(B, B_max_wrt_tau_feas)
    
    print()
    print("B:", B)
    print("B_max_wrt_tau_feas:", B_max_wrt_tau_feas)
    print("B_feas:", B_feas)
    print()

    return np.round(np.maximum(tau_feas.value, 0.0), decimals=4), \
            np.round(np.maximum(B_feas, 0.0), decimals=4)
    
#     return tau_feas.value, B_feas



In [33]:
# # OLD def proj_tau_B:

# def proj_tau_B(T, num_edges, tau, B, od_to_edges_list_full, tau_max, B_max = 1.0):

# #     print()
# #     print("tau.shape[0]:", tau.shape[0])
# #     print("num_edges:", num_edges)
# #     print()

#     assert tau.shape[0] == num_edges, "tau must have length equal to the number of edges."
#     assert tau_max.shape[0] == num_edges, "tau_max must have length equal to the number of edges."
    
#     tau_feas = cp.Variable((num_edges, T))
#     B_feas = cp.Variable(1)
    
#     func = cp.sum_squares(tau_feas - tau) + (B_feas - B)**2

#     objective = cp.Minimize(func)

#     constraints = []
#     constraints += [tau_feas >= 0.0]
#     constraints += [B_feas >= 0.0]
#     constraints += [tau_feas <= tau_max.reshape((num_edges, 1)) * np.ones((1, T))]
#     constraints += [B_feas <= B_max]

#     constraints += [B_feas <= sum(tau_feas[e, t] for e in od_to_edges_list_full[od]) \
#                     for od in range(len(od_to_edges_list_full)) for t in range(T)]
    
#     prob = cp.Problem(objective, constraints)
#     result = prob.solve()

# #     print()
# #     print("tau_feas.value:", np.round(np.maximum(tau_feas.value, 0.0), decimals=4))
# #     print()

#     return np.round(np.maximum(tau_feas.value, 0.0), decimals=4), \
#             np.round(np.maximum(B_feas.value, 0.0), decimals=4)

# Chinmay's Algorithm:

## Convex Program for CBCP and DBCP Equilibria:

In [34]:
## Steps
# 1: Define variables
# 2: Define objective
# 3: Define constraints
# 4: Define problem
# 5: Solve problem
# 6: Extract values

In [35]:
# num_edges = 8
# od_to_edges_array = np.array([[0, 0], [0, 3], [0, 7], [1, 5], [1, 7], [3, 3], [3, 6]])

# od_to_edges_list_full = [list(range(od_to_edges_array[od, 0], od_to_edges_array[od, 1] + 1)) \
#                       for od in range(od_to_edges_array.shape[0])]

# edge_to_od = [[od for od in range(od_to_edges_array.shape[0]) if e in od_to_edges_list_full[od]] \
#             for e in range(num_edges)]

# print(od_to_edges_list_full)
# print()
# print(edge_to_od)

In [36]:

# Below: For quartic latency functions:
# Latency Function: a_4 x^4 + a_3 x^3 + a_2 x^2 + a_1 x + a_0

def solve_CBCP_direct(T, num_edges, num_gp_lanes, tau, B, od_to_edges_array, \
                      demand_array, VoT_array, num_el, coeff_input):
    
#     print("tau (in solve_CBCP_direct):", tau)
    
    coeff = coeff_from_input(coeff_input, num_edges, num_gp_lanes)
    
    assert VoT_array.shape[2] == T, "We should have VoT_array.shape[2] == T."
    assert demand_array.shape == VoT_array[:, :, 0].shape, "demand_array and VoT_array[:, :, 0] should have identical shape."
    assert np.all(demand_array > 0.0), "Each entry of demand_array must be strictly positive."
    assert np.all(tau >= 0.0), "Each entry of tau must be non-negative."
    assert num_el <= demand_array.shape[1], "num_el, the number of eligible income groups, should not exceed \
                                            demand_array.shape[1], which should equal the number of income groups."
    
    num_groups = demand_array.shape[1]
    
    ## Variable indices:
    # y indices: (od, income group, edge, "lane", time)
    # y indices: (od, income group, edge, "lane", time)

    el_indices = list(range(num_el))
    in_indices = list(range(num_el, num_groups))
    group_indices = list(range(num_groups))
    
    od_to_edges_list_full = [list(range(int(od_to_edges_array[od, 0]), int(od_to_edges_array[od, 1]) + 1)) \
                          for od in range(od_to_edges_array.shape[0])]
    
    edge_to_ods = [[od for od in range(od_to_edges_array.shape[0]) if e in od_to_edges_list_full[od]] \
                   for e in range(num_edges)]
    
    num_od = len(od_to_edges_list_full)
    
    # Variables:
    y = {}
    for od in range(num_od):
        for e in od_to_edges_list_full[od]:
            for t in range(T):
                for g in el_indices:
                    for k in [0, 1, 2]:
                        y[(od, g, e, k, t)] = cp.Variable(1)
                for g in in_indices:
                    for k in [0, 1]:
                        y[(od, g, e, k, t)] = cp.Variable(1)
    
    x = {}
    for e in range(num_edges):
        for k in [0, 1]:
            for t in range(T):
                x[(e, k, t)] = cp.Variable(1)

    # Objective:
    func = 0.0
    for e in range(num_edges):
        for t in range(T):
            func += coeff[0, e, 0] * x[e, 0, t] \
                    + 0.5 * coeff[1, e, 0] * cp.square(cp.maximum(x[e, 0, t] - coeff[2, e, 0], 0))
            func += coeff[0, e, 1] * x[e, 1, t] \
                    + 0.5 * coeff[1, e, 1] * cp.square(cp.maximum(x[e, 1, t]/num_gp_lanes - coeff[2, e, 1], 0))
    
#     print("VoT_array.shape:", VoT_array.shape)
#     print("el_indices:", el_indices)
#     print("in_indices:", in_indices)
    
    for od in range(num_od):
        for e in od_to_edges_list_full[od]:
            for t in range(T):
                for g in el_indices:
                    func += tau[e, t] * y[(od, g, e, 1, t)] / VoT_array[od, g, t]
                for g in in_indices:
                    func += tau[e, t] * y[(od, g, e, 0, t)] / VoT_array[od, g, t]

    objective = cp.Minimize(func)
    
    # Constraints:
    constraints = []
    
    constraints += [y[(od, g, e, k, t)] >= 0.0 for od in range(num_od) \
                    for e in od_to_edges_list_full[od] for g in el_indices  \
                    for k in [0, 1, 2] for t in range(T)]
    constraints += [y[(od, g, e, k, t)] >= 0.0 for od in range(num_od) \
                    for e in od_to_edges_list_full[od] for g in in_indices  \
                    for k in [0, 1] for t in range(T)]
    constraints += [x[(e, k, t)] >= 0.0 for e in range(num_edges) \
                    for k in [0, 1] for t in range(T)]
    
#     print()
#     print("demand_array:", demand_array)
#     print()
#     print("B:", B)
#     print()
#     print("el_indices:", el_indices)
#     print()
#     print("in_indices:", in_indices)
#     print()
#     print("edge_to_ods:", edge_to_ods)
#     print()
#     print("od_to_edges_list_full:", od_to_edges_list_full)
#     print()
    
    od_g_e_t_list = []

    for e in range(num_edges):
        for t in range(T):
            
            ## Edge contraints:
            constraints += [sum( y[(od, g, e, 0, t)] + y[(od, g, e, 1, t)] for od in edge_to_ods[e] for g in el_indices) \
                                + sum( y[(od, g, e, 0, t)] for od in edge_to_ods[e] for g in in_indices ) \
                                == x[(e, 0, t)] ]
            constraints += [sum( y[(od, g, e, 2, t)] for od in edge_to_ods[e] for g in el_indices) \
                                + sum( y[(od, g, e, 1, t)] for od in edge_to_ods[e] for g in in_indices ) \
                                == x[(e, 1, t)] ]

            ## Group flow constraints:
            for od in edge_to_ods[e]:
                for g in el_indices:
#                     print("(od, g, e, t):", od, g, e, t)
                    assert (od, g, e, t) not in od_g_e_t_list, "Each (od, g, e, t) should occur only once."
                    od_g_e_t_list += [(od, g, e, t)]
        
#                     if (od, g, e, t) == (0, 0, 0, 0):
#                     if (e, t) == (2, 0) or (2, 1) or (2, 3):
#                         constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1, 2]) == demand_array[od, g]]
                    
#                 for g in in_indices:
# #                     print("(od, g, e, t):", od, g, e, t)
#                     assert (od, g, e, t) not in od_g_e_t_list, "Each (od, g, e, t) should occur only once."
#                     od_g_e_t_list += [(od, g, e, t)]
                    
#                     constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1]) == demand_array[od, g]]

            constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1, 2]) == demand_array[od, g] \
                            for od in edge_to_ods[e] for g in el_indices]
            constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1]) == demand_array[od, g] \
                            for od in edge_to_ods[e] for g in in_indices]
    
    constraints += [sum(y[(od, g, e, 0, t)] * tau[e, t] for e in od_to_edges_list_full[od] for t in range(T)) \
                    <= B * demand_array[od, g] for od in range(num_od) for g in el_indices]
    
    # Problem:
    prob = cp.Problem(objective, constraints)
    
    # Solve:
#     result = prob.solve(solver = cp.MOSEK, verbose = True)
    result = prob.solve(solver = cp.MOSEK)
    
#     for variable in prob.variables():
#         print("Variable %s" % variable.name())
    
    assert prob.status != "infeasible", "problem.status should not be infeasible."
    assert prob.status != "unbounded", "problem.status should not be unbounded."
    print()
    print("prob.status:", prob.status)

    # Extract Values:
    y_values = {}
    for e in range(num_edges):
        for od in edge_to_ods[e]:
            for t in range(T):
                for g in el_indices:
                    for k in [0, 1, 2]:
#                         print("(od, g, e, k, t):", od, g, e, k, t)
#                         print("y[(od, g, e, k, t)].value:", y[(od, g, e, k, t)].value)
                        y_values[(od, g, e, k, t)] = max(y[(od, g, e, k, t)].value[0], 0.0)
                for g in in_indices:
                    for k in [0, 1]:
#                         print("(od, g, e, k, t):", od, g, e, k, t)
#                         print("y[(od, g, e, k, t)].value:", y[(od, g, e, k, t)].value)
                        y_values[(od, g, e, k, t)] = max(y[(od, g, e, k, t)].value[0], 0.0)

    return y_values


## <font color='blue'>Implement Zeroth-order Gradient Descent:</font> 

In [37]:
## Functions defined above:

# def proj_tau_B(T, num_edges, tau, B, od_to_edges_list_full, tau_max = 1.0, B_max = 1.0):

# def solve_CBCP_direct(T, num_edges, num_gp_lanes, tau, B, od_to_edges_array, \
#                       demand_array, VoT_array, num_el, coeff_input):

#     def welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, tau, \
#                 demand_array, VoT_array, num_el, od_to_edges_array, y, \
#                 coeff_input = np.array([0.0, 0.0, 0.0, 0.0, 1.0])):

In [38]:
# coeff_input
# demand_array.shape

In [95]:
time_1 = time.time()

T = 5

num_el = 2
num_groups = demand_array.shape[1]

assert VoT_array.shape[2] == T, "We should have VoT_array.shape[2] == T"
assert demand_array.shape == VoT_array[:, :, 0].shape, "demand_array and VoT_array[:, :, 0] should have the same shape."

num_ods = demand_array.shape[0]
group_indices = list(range(demand_array.shape[1]))
# num_edges = 6
num_edges = 7
num_gp_lanes = 3

# tau = np.array([0.2, 0.6, 0.5, 0.8, 0.4, 0.5])
# assert tau.shape[0] == num_edges, "tau must have a num_edges number of entries."

# B = 0.8
a = np.array([1E-3, 1E-3, 0.0, 0.0, 0.0])

error_bound = 1E-2
diffs_num_cols = 5

# lambda_E, lambda_R, lambda_I = 1.0, 1.0, 1.0
lambda_E, lambda_R, lambda_I = 1.0, 1.0, 1.0
# lambda_E, lambda_R, lambda_I = 1.0, 1.5, 1.0

demand_edges_array = np.zeros(num_edges)

# demand_array_temp = np.ones(demand_array.shape)
# demand_array_temp = np.random.uniform(low=0.05, high=0.5, size=demand_array.shape)
demand_array_temp = demand_array


# TODO: Modify coeff_input:

## coeff_input: const, slope, x-coordinate of transition point

# coeff_input = np.array([19.4, 0.1256, 0.786*1650]).reshape((3, 1)) @ np.ones((1, num_edges))


coeff_input = np.zeros((3, num_edges))
for counter, city in enumerate(dict_latency_params.keys()):
    coeff_input[0, counter] = dict_latency_params[city]["Latency (at bend)"]
    coeff_input[1, counter] = dict_latency_params[city]["Slope (after bend)"]
    coeff_input[2, counter] = dict_latency_params[city]["Flow (at bend)"]


# def solve_CBCP_direct(T, num_edges, num_gp_lanes, tau, B, od_to_edges_array, \
#                       demand_array, VoT_array, num_el, coeff_input):
# Already defined: T, num_edges, num_gp_lanes, tau, B, od_to_edges_array, \
#                       demand_array, VoT_array

    
# print("od_to_edges_array[1, 0]:", int(od_to_edges_array[1, 0]))
# print("range(od_to_edges_array[1, 0], od_to_edges_array[1, 1] + 1):",\
#      range(int(od_to_edges_array[1, 0]), int(od_to_edges_array[1, 1]) + 1))

od_to_edges_list_full = [list(range(int(od_to_edges_array[od, 0]), int(od_to_edges_array[od, 1]) + 1)) \
                         for od in range(num_ods)]
edge_to_ods = [[od for od in range(od_to_edges_array.shape[0]) if e in od_to_edges_list_full[od]] \
               for e in range(num_edges)]
for e in range(num_edges):
    demand_edges_array[e] = sum([np.sum(demand_array_temp[od, :]) for od in range(num_ods) \
                                 if od in edge_to_ods[e]])

el_indices = list(range(num_el))
in_indices = list(range(num_el, num_groups))

tau_max_from_latency = np.zeros((num_edges, T))
VoT_max_el = np.zeros((num_edges, T))
for e in range(num_edges):
    for t in range(T):
        VoT_max_el[e, t] = max([VoT_array[od_index, group_index, t] for od_index in edge_to_ods[e] \
                                for group_index in el_indices])
        tau_max_from_latency[e, t] = VoT_max_el[e, t] * (latency_max(demand_edges_array[e], coeff_input[:, e]) - coeff_input[0, e]) \
                                        / num_gp_lanes

tau_max_monetary_value = 15.0
# fraction_tau_max = tau_max_monetary_value / np.max(tau_max_from_latency)

tau_upper_limit = np.minimum(tau_max_from_latency, tau_max_monetary_value)
B_upper_limit = max([sum(tau_upper_limit[e, t] for e in od_to_edges_list_full[od] for t in range(T) ) \
                     for od in range(len(od_to_edges_list_full))])

d = num_edges * T + 1
# num_iters = 1000
# num_iters = 100
num_iters = 200
# num_iters = 3
tau = np.zeros((num_edges, T, num_iters))
tau_perturbed = np.zeros((num_edges, T, num_iters))

B = np.zeros(num_iters)
B_perturbed = np.zeros(num_iters)
delta = np.zeros(num_iters)
eta = np.zeros(num_iters)
eta_bar = 2E-6
delta_bar = 0.01

welfare_list = []

fraction_tau_max = 0.5
fraction_B_max = 0.5

# tau[:, :, 0] = fraction_tau_max * tau_upper_limit.reshape((num_edges, 1)) @ np.ones((1, T))
tau[:, :, 0] = np.random.triangular(np.zeros((num_edges, T)), tau_upper_limit * fraction_tau_max, tau_upper_limit)
# B[0] = fraction_B_max * B_upper_limit
B[0] = np.random.triangular(0.0, fraction_B_max * B_upper_limit, B_upper_limit)

od_to_edges_list_full = [list(range(int(od_to_edges_array[od, 0]), int(od_to_edges_array[od, 1]) + 1 )) \
                         for od in range(od_to_edges_array.shape[0])]

# print("od_to_edges_list_full:", od_to_edges_list_full)
# print("tau_max_from_latency:", tau_max_from_latency)
# print("B_max", B_max)


# # To disable when restarting from scratch
# tau[:, :, 0] = tau_next_init
# B[0] = B_next_init
# eta[0] = eta_bar * (index_next_init+1)**(-1/2) * d**(-1)
# delta[0] = delta_bar * (index_next_init+1)**(-1/4) * d**(-1/2)

for i in range(num_iters-1):
    
    print()
    print("Iter:", i)
    
    eta[i] = eta_bar * (i+1)**(-1/2) * d**(-1)
    delta[i] = delta_bar * (i+1)**(-1/4) * d**(-1/2)
    w_i_unnormalized = np.random.randn(d)
    w_i = w_i_unnormalized / np.linalg.norm(w_i_unnormalized)
#     print("w_i:", w_i)
    tau_perturbed[:, :, i] = tau[:, :, i] + delta[i] * w_i[:-1].reshape((num_edges, T))
    B_perturbed[i] = B[i] + delta[i] * w_i[-1]
    
#     if tau_perturbed[:, :, i] < B_perturbed[i] or tau_perturbed[:, :, i] < 0 or B_perturbed[i] < 0:

    print("i:\n", i)
    print("tau[:, :, i], before projection:\n", tau[:, :, i])
    print("B[i], before projection:\n", B[i])
    print("tau_perturbed[:, :, i], before projection:\n", tau_perturbed[:, :, i])
    print("B_perturbed[i], before projection:\n", B_perturbed[i])

    tau_perturbed[:, :, i], B_perturbed[i] = proj_tau_B(T, num_edges, tau_perturbed[:, :, i], B_perturbed[i], \
                                                        od_to_edges_list_full, \
                                                        tau_max = tau_upper_limit, B_max = B_upper_limit)
    
    print("tau[:, :, i], after projection:\n", tau[:, :, i])
    print("B[i], after projection:\n", B[i])
    print("tau_perturbed[:, :, i], after projection:\n", tau_perturbed[:, :, i])
    print("B_perturbed[i], after projection:\n", B_perturbed[i])

    # TODO: Remove "network":

    y_values = solve_CBCP_direct(T, num_edges, num_gp_lanes, \
                                 tau[:, :, i], B[i], od_to_edges_array, \
                                 demand_array_temp, VoT_array, num_el, coeff_input)

    y_perturbed_values = solve_CBCP_direct(T, num_edges, num_gp_lanes, \
                                           tau_perturbed[:, :, i], B_perturbed[i], od_to_edges_array, \
                                           demand_array_temp, VoT_array, num_el, coeff_input)
    
#     def welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, tau, \
#                 demand_array, VoT_array, num_el, od_to_edges_array, y, \
#                 coeff_input):

    welfare, obj_E, obj_R, obj_I = welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, \
                                               tau[:, :, i], demand_array_temp, VoT_array, num_el, od_to_edges_array, \
                                               y = y_values, coeff_input = coeff_input)
    
    welfare_perturbed, obj_E_perturbed, obj_R_perturbed, obj_I_perturbed \
        = welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, \
                      tau_perturbed[:, :, i], demand_array_temp, VoT_array, num_el, od_to_edges_array, \
                      y = y_perturbed_values, coeff_input = coeff_input)
    
    print("welfare:", welfare)
    print("obj_E:", obj_E)
    print("obj_R:", obj_R)
    print("obj_I:", obj_I)
    print()
    print("welfare_perturbed:", welfare_perturbed)
    print("obj_E_perturbed:", obj_E_perturbed)
    print("obj_R_perturbed:", obj_R_perturbed)
    print("obj_I_perturbed:", obj_I_perturbed)
    print()
    
    welfare_list.append(welfare)
    
    tau[:, :, i+1] = tau[:, :, i] - eta[i] * (d/delta[i]) * w_i[:-1].reshape((num_edges, T)) \
                        * (welfare_perturbed - welfare)
    
    B[i+1] = B[i] - eta[i] * (d/delta[i]) * w_i[-1] * (welfare_perturbed - welfare)
    
    print("tau[:, :, i+1], before projection:\n", tau[:, :, i+1])
    print("B[i+1], before projection:\n", B[i+1])
    
    tau[:, :, i+1], B[i+1] = proj_tau_B(T, num_edges, tau[:, :, i+1], B[i+1], od_to_edges_list_full, \
                                        tau_max = tau_upper_limit, B_max = B_upper_limit)
    
    print("tau[:, :, i+1], after projection:\n", tau[:, :, i+1])
    print("B[i+1], after projection:\n", B[i+1])
    
    if i >= diffs_num_cols + 2:
        tau_diffs = np.max(np.absolute(tau[:, :, i-diffs_num_cols : i-1] - tau[:, :, i-diffs_num_cols+1 : i]), axis = 2)
        B_diffs = B[i-diffs_num_cols : i-1] - B[i-diffs_num_cols+1 : i]
        
#         print("tau[:, :, 0:10]:", tau[:, :, 0:10])
#         print("B[0:10]:", B[0:10])
        print("tau_diffs:\n", tau_diffs)
        print("B_diffs:\n", B_diffs)
        print()
        
        if max(np.max(np.absolute(tau_diffs)), np.max(np.absolute(B_diffs))) < error_bound:
            print("Within error bound.")
            break

time_2 = time.time()

print()
print("Time:", time_2 - time_1)




Iter: 0
i:
 0
tau[:, :, i], before projection:
 [[0.08025705 0.00605028 0.07248283 0.00373566 0.07648759]
 [0.0430071  0.04736999 0.04427704 0.04893302 0.04398793]
 [0.19957439 0.29310759 0.09101882 0.22661591 0.11080917]
 [0.23904261 0.22850079 0.23781002 0.06571522 0.26619194]
 [0.24442065 0.01677287 0.06951079 0.11803252 0.09497929]
 [0.03079321 0.05147023 0.03116595 0.02626001 0.03749177]
 [0.06731163 0.04069861 0.00160528 0.04140619 0.01333347]]
B[i], before projection:
 5.608469996959063
tau_perturbed[:, :, i], before projection:
 [[0.08045438 0.00527217 0.0727129  0.00391483 0.07663734]
 [0.04302015 0.0476688  0.04407561 0.04908951 0.0439538 ]
 [0.19972492 0.29290799 0.09124746 0.22622241 0.11046496]
 [0.23915256 0.22848676 0.23767024 0.06586708 0.26562545]
 [0.24477084 0.01677733 0.06950393 0.11786376 0.09524985]
 [0.03037392 0.05176434 0.03117997 0.02615727 0.03747323]
 [0.06675968 0.04057237 0.00182725 0.04154751 0.01286578]]
B_perturbed[i], before projection:
 5.60847206069


prob.status: optimal

prob.status: optimal

in_indices: [2, 3, 4]
el_indices: [0, 1]


in_indices: [2, 3, 4]
el_indices: [0, 1]

welfare: 743243.3088024318
obj_E: 5972.4928973836495
obj_R: 5556.335551685292
obj_I: 742827.1514567335

welfare_perturbed: 753437.5124678157
obj_E_perturbed: 6073.479006109461
obj_R_perturbed: 5555.788160847736
obj_I_perturbed: 752919.821622554

tau[:, :, i+1], before projection:
 [[-0.98245417 -0.80724211  1.84477497 -0.87392214 -2.47817874]
 [ 0.64498492  0.94573427  1.40171524  2.34963781  2.39653523]
 [ 1.69730347  1.54362913 -0.88032123 -0.78170114  2.27358578]
 [-1.4542994  -0.76309735  1.90093996 -0.98687681  0.42101784]
 [-0.37994415  0.99687245  2.04213603  3.32999738 -1.36514519]
 [-2.69382488 -0.98415818  0.0945975  -2.13539666  2.21273887]
 [ 0.38368889  0.92116723 -1.53247808  1.01386597 -1.17568209]]
B[i+1], before projection:
 0.10374252503963877

B: 0.10374252503963877
B_max_wrt_tau_feas: 4.158021101535954
B_feas: 0.10374252503963877

tau[:, 


prob.status: optimal

prob.status: optimal

in_indices: [2, 3, 4]
el_indices: [0, 1]


in_indices: [2, 3, 4]
el_indices: [0, 1]

welfare: 736438.0982547628
obj_E: 5915.263153303563
obj_R: 5918.618689651741
obj_I: 736441.453791111

welfare_perturbed: 738060.474791736
obj_E_perturbed: 5932.640309441666
obj_R_perturbed: 5917.761891087824
obj_I_perturbed: 738045.5963733821

tau[:, :, i+1], before projection:
 [[ 0.27696458  0.11019826  0.19490764  0.04526518 -0.20589414]
 [ 0.23702068 -0.05963145 -0.41342162  0.20040487  0.00813761]
 [ 0.39547981  0.32110108  0.33959829  0.09829243  0.10298343]
 [-0.07054748  0.42402493  0.29284429 -0.3011941   0.32626602]
 [-0.00471888 -0.04077753  0.52617227  0.20598751  1.10684654]
 [-0.13103799  0.19878197  0.30849386 -0.01466101  0.00216982]
 [ 0.00660559 -0.01696291 -0.02450236 -0.09147406 -0.00643151]]
B[i+1], before projection:
 0.014786845899465523

B: 0.014786845899465523
B_max_wrt_tau_feas: 4.093763553212073
B_feas: 0.014786845899465523

tau[:,


prob.status: optimal

prob.status: optimal

in_indices: [2, 3, 4]
el_indices: [0, 1]


in_indices: [2, 3, 4]
el_indices: [0, 1]

welfare: 732877.4302060996
obj_E: 5855.232011767382
obj_R: 6751.627096503947
obj_I: 733773.8252908362

welfare_perturbed: 737665.3778012212
obj_E_perturbed: 5880.754719488678
obj_R_perturbed: 6750.981660340514
obj_I_perturbed: 738535.6047420731

tau[:, :, i+1], before projection:
 [[ 0.62678349  0.31219939 -0.17179404 -0.20660061  0.97502516]
 [ 0.28289956  0.24353452 -0.24561814 -1.18696328  0.1426447 ]
 [ 0.40442173 -0.367015   -0.99018487  0.96788896 -0.47401112]
 [ 0.24521695  0.97853212 -1.28014947 -0.58018393  0.79098182]
 [ 0.64015303  0.14846589  0.65887383  0.23126183  1.13202605]
 [ 0.35866634  0.97635695 -0.17173577  0.06182084  0.37872219]
 [ 0.25860935  0.19195559  0.69458595 -0.3165549   0.44620827]]
B[i+1], before projection:
 -0.5307152076812717

B: -0.5307152076812717
B_max_wrt_tau_feas: 4.694695700806386
B_feas: -0.5307152076812717

tau[:, 


prob.status: optimal

prob.status: optimal

in_indices: [2, 3, 4]
el_indices: [0, 1]


in_indices: [2, 3, 4]
el_indices: [0, 1]

welfare: 749196.6687943687
obj_E: 6022.84387548825
obj_R: 4713.486542583266
obj_I: 747887.3114614638

welfare_perturbed: 754003.6502046011
obj_E_perturbed: 6108.495784798253
obj_R_perturbed: 4761.939992381432
obj_I_perturbed: 752657.0944121843

tau[:, :, i+1], before projection:
 [[ 0.85989691  0.59765708 -0.01061953 -0.1695061  -0.73725992]
 [-0.43631002 -0.11853603  0.24605173 -0.47245801  0.22133306]
 [-0.63003184  0.2882395  -0.2522614   1.10420533 -0.9907829 ]
 [ 0.49995912  0.94976827 -0.22328664  0.3557058   1.05839161]
 [-0.63688202  0.8739276   0.03320104 -1.19831622 -0.01447245]
 [ 0.25625384 -0.57304404  0.04277288  0.03512602 -0.12885204]
 [ 0.20532286  0.41188785 -0.08810869  0.59558898  0.39505395]]
B[i+1], before projection:
 -0.005181815918242461

B: -0.005181815918242461
B_max_wrt_tau_feas: 3.3720407643567016
B_feas: -0.005181815918242461

t

...


prob.status: optimal

prob.status: optimal

in_indices: [2, 3, 4]
el_indices: [0, 1]


in_indices: [2, 3, 4]
el_indices: [0, 1]

welfare: 786482.8881100862
obj_E: 6102.304168652475
obj_R: 3266.653944752353
obj_I: 783647.2378861861

welfare_perturbed: 788732.6586357465
obj_E_perturbed: 6119.570817035244
obj_R_perturbed: 3266.136280987164
obj_I_perturbed: 785879.2240996984

tau[:, :, i+1], before projection:
 [[-0.04110852 -0.14556714 -0.16233763  0.11063305  0.04926265]
 [-0.2263804   0.19422743  0.04941824  0.02738463 -0.07099375]
 [ 0.44391723 -0.02420092  0.28181575 -0.13673585  0.4283494 ]
 [ 0.31714023  0.20603716  0.21544875  0.56248954  0.12157731]
 [ 0.21561898  0.08823478  0.65858104  0.70035152  0.4260921 ]
 [-0.0885073   0.25741782  0.21450334  0.23685235  0.22195309]
 [ 0.10051603  0.37543941  0.14544495 -0.1567071  -0.04995252]]
B[i+1], before projection:
 3.823431071515232

B: 3.823431071515232
B_max_wrt_tau_feas: 4.740490137679289
B_feas: 3.823431071515232

tau[:, :, i+1


prob.status: optimal

prob.status: optimal

in_indices: [2, 3, 4]
el_indices: [0, 1]


in_indices: [2, 3, 4]
el_indices: [0, 1]

welfare: 776869.304189556
obj_E: 6030.924694835614
obj_R: 2762.8197867925664
obj_I: 773601.199281513

welfare_perturbed: 805139.7853357757
obj_E_perturbed: 6213.39947660134
obj_R_perturbed: 2763.5993322045556
obj_I_perturbed: 801689.9851913789

tau[:, :, i+1], before projection:
 [[ 0.17195735  5.16913786 -2.66859313  2.0910738  -0.51320543]
 [ 1.03745863  2.75265964 -0.35060311 -2.36681889  2.67347092]
 [ 1.70635295  2.7885854  -1.31929535 -1.59761708 -2.81022952]
 [-1.47566692  0.32960588  0.57015206 -3.91417887 -3.78976453]
 [-0.01647297  0.88334439  2.65438416 -0.12063058 -1.87625245]
 [-2.23942763  2.40627292 -2.05909218 -0.68451463 -3.60176336]
 [ 0.38618548  1.18361151 -0.15585565 -1.67542629 -1.56706649]]
B[i+1], before projection:
 5.469577084066157

B: 5.469577084066157
B_max_wrt_tau_feas: 3.1867119839807327
B_feas: 3.1867119839807327

tau[:, :, i+


prob.status: optimal

prob.status: optimal

in_indices: [2, 3, 4]
el_indices: [0, 1]


in_indices: [2, 3, 4]
el_indices: [0, 1]

welfare: 786279.3786417835
obj_E: 6108.205465792395
obj_R: 3309.249472354273
obj_I: 783480.4226483454

welfare_perturbed: 803181.3834936452
obj_E_perturbed: 6243.877699276638
obj_R_perturbed: 3309.460074672067
obj_I_perturbed: 800246.9658690407

tau[:, :, i+1], before projection:
 [[-1.38296803  1.39219876  1.82288388 -0.18883784  1.53141537]
 [-0.69201679  0.19877953  1.78279733  0.63495342 -0.95099796]
 [ 0.89419562 -0.97277699 -0.38082377 -1.67238969 -2.754501  ]
 [ 1.43116193  1.12941028  0.57789672  1.55791126  1.34314622]
 [ 1.8829466  -0.74123076 -2.63932272 -1.00296391  0.29530387]
 [-0.26144898 -1.51074415  1.00162522  0.04991116 -0.80360677]
 [ 0.10152138  0.42587806  0.5726979   0.93370588  2.19715492]]
B[i+1], before projection:
 3.6385669172089834

B: 3.6385669172089834
B_max_wrt_tau_feas: 3.922963943407085
B_feas: 3.6385669172089834

tau[:, :, 


prob.status: optimal

prob.status: optimal

in_indices: [2, 3, 4]
el_indices: [0, 1]


in_indices: [2, 3, 4]
el_indices: [0, 1]

welfare: 761374.038376189
obj_E: 5960.3840213836065
obj_R: 2114.6152061431094
obj_I: 757528.2695609485

welfare_perturbed: 771945.0914596599
obj_E_perturbed: 6020.715196129821
obj_R_perturbed: 2114.934542783457
obj_I_perturbed: 768039.3108063135

tau[:, :, i+1], before projection:
 [[ 0.26052079 -1.59057852  0.45600365 -0.54692745 -0.83522894]
 [ 0.94699756  0.36943788  0.67665441  0.20875983  1.64926632]
 [ 1.04665626  1.15575947  0.25753762  0.91053833  0.62373675]
 [ 0.59158387  0.19421289  1.42879488 -0.24309418  1.48205887]
 [-0.68143994 -0.36364253  0.60547161 -0.01739127 -1.42306401]
 [-0.04017888  0.43459186  0.96029044  0.03494385 -0.43603035]
 [-0.47381726 -0.29759017  0.16516423 -0.56459828 -0.32409584]]
B[i+1], before projection:
 2.0257487243160828

B: 2.0257487243160828
B_max_wrt_tau_feas: 4.212327488707639
B_feas: 2.0257487243160828

tau[:, :,


prob.status: optimal

prob.status: optimal

in_indices: [2, 3, 4]
el_indices: [0, 1]


in_indices: [2, 3, 4]
el_indices: [0, 1]

welfare: 753867.9041763871
obj_E: 6048.65096441148
obj_R: 4981.289777017065
obj_I: 752800.5429889928

welfare_perturbed: 759252.8365520944
obj_E_perturbed: 6102.211795530291
obj_R_perturbed: 4982.999198457059
obj_I_perturbed: 758133.6239550211

tau[:, :, i+1], before projection:
 [[-0.01729562  0.29350227  0.00545521  0.01587726 -0.01893379]
 [-0.50918466  0.39866207 -0.31133597 -0.22186508 -0.49721284]
 [-0.61170668 -0.23545235 -0.421613    0.46284548 -0.06278171]
 [ 0.086852    0.11131878  0.47664861  0.29412868  0.0295905 ]
 [-0.0382671   0.65372875  0.84754893  0.16673226  0.04107605]
 [ 0.08859319 -0.65291253 -0.1032811  -0.14944425 -0.39103145]
 [ 0.08545839  0.63272183 -0.74694534 -0.32092685 -0.04540795]]
B[i+1], before projection:
 2.1467419588224965

B: 2.1467419588224965
B_max_wrt_tau_feas: 2.89545577481322
B_feas: 2.1467419588224965

tau[:, :, i+

In [ ]:
# dict_latency_params["Belmont"]

In [96]:
min_welfare = min(welfare_list)
argmin_welfare_list = welfare_list.index(min(welfare_list))
argmin_tau = tau[:, :, argmin_welfare_list]
argmin_B = B[argmin_welfare_list]

print("argmin_B:", argmin_B)
print("B[argmin_welfare_list]:", B[argmin_welfare_list])

print(welfare_list)
B

argmin_B: 0.0
B[argmin_welfare_list]: 0.0
[815361.5827674664, 772799.6083358871, 743243.3088024318, 733047.6485137532, 738818.5549115649, 736438.0982547628, 734403.6545410688, 731118.2419971655, 732877.4302060996, 734131.2578650566, 734225.2006062509, 749196.6687943687, 731255.2553297016, 735277.5317745522, 735157.7722942615, 738090.4019238366, 738301.1860525234, 738356.7574913818, 736240.5511903879, 732216.5759762228, 731915.2364981939, 738727.3280014232, 732922.9333192466, 733949.107295969, 735883.4988527925, 735367.8670429213, 739880.4030258829, 742968.095915769, 735876.833775447, 738345.3240735249, 737055.4680990889, 735793.2302221683, 739393.664376592, 736177.9726860758, 737373.3185479782, 754574.142716486, 741142.5567308019, 751058.3040532524, 749338.7910177825, 774277.5346095667, 771690.2269351622, 790140.0827964781, 780067.944204966, 790274.3970467928, 786482.8881100862, 801275.7385652475, 798961.3915724466, 776869.304189556, 767040.549678255, 785239.3276143229, 786279.37864178

array([5.60847, 3.3481 , 0.8058 , 0.1037 , 0.2795 , 0.1525 , 0.0148 ,
       0.     , 0.     , 0.     , 0.     , 0.3227 , 0.     , 0.     ,
       0.     , 0.008  , 0.0658 , 0.0487 , 0.0295 , 0.     , 0.     ,
       0.3679 , 0.     , 0.     , 0.0433 , 0.     , 0.1598 , 0.4721 ,
       0.0976 , 0.2038 , 0.0974 , 0.1968 , 0.1789 , 0.1526 , 0.1952 ,
       0.8684 , 0.7315 , 1.0222 , 1.1142 , 2.6979 , 2.5512 , 3.1806 ,
       3.1792 , 3.1409 , 3.4842 , 3.8234 , 3.6152 , 3.3659 , 3.1867 ,
       2.9492 , 3.3121 , 3.6386 , 3.7491 , 2.774  , 2.0257 , 2.0822 ,
       1.137  , 2.1467 , 2.4047 , 2.2696 ])

In [44]:
index_next_init = argmin_welfare_list
B_next_init = B[index_next_init]
tau_next_init = tau[:, :, index_next_init]

print("index_next_init:", index_next_init)
print("B_next_init:", B_next_init)
print("tau_next_init:", tau_next_init)


index_next_init: 124
B_next_init: 0.0
tau_next_init: [[0.     0.     0.     0.1065 0.    ]
 [0.0833 0.     0.0833 0.0833 0.    ]
 [0.3332 0.     0.     0.     0.    ]
 [0.3427 0.     0.     0.3427 0.    ]
 [0.     0.     0.     0.     0.    ]
 [0.0641 0.0641 0.     0.0641 0.    ]
 [0.0726 0.0726 0.     0.0726 0.    ]]


In [ ]:
# lambda combinations to try to simulate:
# (1, 1, 1)
# (5, 5, 1)
# (10, 10, 1)
# (5, 10, 1)
# (5, 15, 1)
# (10, 15, 1)
# (10, 20, 1)
# (20, 20, 1)
# (20, 30, 1)
# (20, 40, 1)

In [ ]:

# argmin_tau = np.array([[0.7091, 1.8252, 0.3238, 0.1215, 0.1079],
#                        [1.8739, 0.7429, 0.0, 0.9112, 1.8739],
#                        [0.0, 3.0448, 0.0, 0.0, 3.9494],
#                        [2.9504, 2.377, 3.4944, 0.0, 0.0],
#                        [7.2392, 8.4126, 0.0, 6.819, 0.6135],
#                        [0.2836, 0.8496, 0.963, 0.9527, 0.5689],
#                        [0.1346, 0.0, 0.0, 0.0712, 0.5024]])

# argmin_B = 0.1664

In [97]:
argmin_y = solve_CBCP_direct(T, num_edges, num_gp_lanes, \
                                    argmin_tau, argmin_B, od_to_edges_array, \
                                    demand_array_temp, VoT_array, num_el, coeff_input)

print("argmin_tau:", argmin_tau)
print("argmin_B:", argmin_B)
print("argmin_y:\n")

argmin_y



prob.status: optimal
argmin_tau: [[0.1065 0.1065 0.1065 0.1065 0.    ]
 [0.0118 0.0833 0.0833 0.0833 0.    ]
 [0.     0.     0.3332 0.3332 0.    ]
 [0.     0.3427 0.     0.0678 0.3427]
 [0.     0.     0.5348 0.     0.5348]
 [0.0641 0.     0.0641 0.0641 0.    ]
 [0.0726 0.     0.     0.     0.0726]]
argmin_B: 0.0
argmin_y:



{(0, 0, 0, 0, 0): -0.0,
 (0, 0, 0, 1, 0): -0.0,
 (0, 0, 0, 2, 0): 47.77809012103125,
 (0, 1, 0, 0, 0): -0.0,
 (0, 1, 0, 1, 0): 0.0001619469302747985,
 (0, 1, 0, 2, 0): 27.564120815203133,
 (0, 2, 0, 0, 0): 0.0003998718989344305,
 (0, 2, 0, 1, 0): 115.15704811212508,
 (0, 3, 0, 0, 0): 0.0012729444768416437,
 (0, 3, 0, 1, 0): 134.7574427815087,
 (0, 4, 0, 0, 0): 0.1334188415684033,
 (0, 4, 0, 1, 0): 287.1476615015553,
 (0, 0, 0, 0, 1): -0.0,
 (0, 0, 0, 1, 1): -0.0,
 (0, 0, 0, 2, 1): 47.77809012103125,
 (0, 1, 0, 0, 1): -0.0,
 (0, 1, 0, 1, 1): 0.00016219868837389945,
 (0, 1, 0, 2, 1): 27.56412056344503,
 (0, 2, 0, 0, 1): 0.0005772767818885544,
 (0, 2, 0, 1, 1): 115.15687070724213,
 (0, 3, 0, 0, 1): 0.0010890895271009758,
 (0, 3, 0, 1, 1): 134.75762663645844,
 (0, 4, 0, 0, 1): 5.160412065627984,
 (0, 4, 0, 1, 1): 282.1206682774957,
 (0, 0, 0, 0, 2): -0.0,
 (0, 0, 0, 1, 2): -0.0,
 (0, 0, 0, 2, 2): 47.77809012103125,
 (0, 1, 0, 0, 2): -0.0,
 (0, 1, 0, 1, 2): 0.00016212329881361826,
 (0, 1, 0

In [54]:
# demand_array_temp

# tau_upper_limit

B_upper_limit

# y_perturbed_values = solve_CBCP_direct(T, num_edges, num_gp_lanes, \
#                                            tau_perturbed[:, :, i], B_perturbed[i], od_to_edges_array, \
#                                            demand_array_temp, VoT_array, num_el, coeff_input)

# T
# num_edges
# num_gp_lanes
# B_perturbed[i]
# coeff_input
# od_to_edges_array
# demand_array_temp
# VoT_array
# num_el
# coeff_input

# np.minimum(np.array([1, 2]), 0.5)

# tau_upper_limit
# argmin_welfare_list

7.685308100508504

# Compute Cost Metrics

In [98]:
num_groups = demand_array.shape[1]

el_indices = list(range(num_el))
in_indices = list(range(num_el, num_groups))

edge_to_od_dict = {}
for e in range(num_edges):
    edge_to_od_dict[e] = [k for k in list(range(int(od_to_edges_array.shape[0]) )) \
                           if od_to_edges_array[k, 0] <= e <= od_to_edges_array[k, 1]]

argmin_x = {}
for e in range(num_edges):
    for t in range(T):
        argmin_x[(e, 0, t)] = 0
        argmin_x[(e, 0, t)] += sum(argmin_y[(od, g, e, 0, t)] + argmin_y[(od, g, e, 1, t)] \
                                   for od in edge_to_od_dict[e] for g in el_indices)
        argmin_x[(e, 0, t)] += sum(argmin_y[(od, g, e, 0, t)] \
                                   for od in edge_to_od_dict[e] for g in in_indices)
        
        argmin_x[(e, 1, t)] = 0
        argmin_x[(e, 1, t)] += sum(argmin_y[(od, g, e, 2, t)] \
                                   for od in edge_to_od_dict[e] for g in el_indices)
        argmin_x[(e, 1, t)] += sum(argmin_y[(od, g, e, 1, t)] \
                                   for od in edge_to_od_dict[e] for g in in_indices)

argmin_y_in_el_total = {}      
for e in range(num_edges):
    for t in range(T):
        argmin_y_in_el_total[(e, 0, t, 'el')] = sum(argmin_y[(od, g, e, 0, t)] + argmin_y[(od, g, e, 1, t)] \
                                                    for od in edge_to_od_dict[e] for g in el_indices)
        
        argmin_y_in_el_total[(e, 1, t, 'el')] = sum(argmin_y[(od, g, e, 2, t)] \
                                                    for od in edge_to_od_dict[e] for g in el_indices)
        
        argmin_y_in_el_total[(e, 0, t, 'in')] = sum(argmin_y[(od, g, e, 0, t)] \
                                                    for od in edge_to_od_dict[e] for g in in_indices)
        
        argmin_y_in_el_total[(e, 1, t, 'in')] = sum(argmin_y[(od, g, e, 1, t)] \
                                                    for od in edge_to_od_dict[e] for g in in_indices)


# y[(od, g, e, 0, t)] + y[(od, g, e, 1, t)] for od in edge_to_od_dict[e] for g in el_indice

# argmin_x

argmin_y_in_el_total

{(0, 0, 0, 'el'): 0.00021476409326601052,
 (0, 1, 0, 'el'): 596.3165320323733,
 (0, 0, 0, 'in'): 1154.4567343589165,
 (0, 1, 0, 'in'): 3097.330151336215,
 (0, 0, 1, 'el'): 0.0002150393848633476,
 (0, 1, 1, 'el'): 596.3165317570817,
 (0, 0, 1, 'in'): 1152.3708952014417,
 (0, 1, 1, 'in'): 3099.4159904936896,
 (0, 0, 2, 'el'): 0.0002149617693283852,
 (0, 1, 2, 'el'): 596.3165318346972,
 (0, 0, 2, 'in'): 1153.6541548167722,
 (0, 1, 2, 'in'): 3098.132730878359,
 (0, 0, 3, 'el'): 0.00021424795581347002,
 (0, 1, 3, 'el'): 596.3165325485107,
 (0, 0, 3, 'in'): 1156.8921974890338,
 (0, 1, 3, 'in'): 3094.8946882060973,
 (0, 0, 4, 'el'): 0.0,
 (0, 1, 4, 'el'): 596.3167467964665,
 (0, 0, 4, 'in'): 1212.048861331159,
 (0, 1, 4, 'in'): 3039.738024363972,
 (1, 0, 0, 'el'): 0.00359594997391883,
 (1, 1, 0, 'el'): 658.9548022180935,
 (1, 0, 0, 'in'): 1285.0419798974388,
 (1, 1, 0, 'in'): 3230.230469154867,
 (1, 0, 1, 'el'): 0.0005343471667647077,
 (1, 1, 1, 'el'): 658.9578638209007,
 (1, 0, 1, 'in'): 123

In [99]:
# coeff_input

print("argmin_tau[5, 3]:", argmin_tau[5, 3])
print("argmin_x[(5, 0, 3)]:", argmin_x[(5, 0, 3)])
print("argmin_x[(5, 1, 3)]:", argmin_x[(5, 1, 3)])


argmin_tau[5, 3]: 0.0641
argmin_x[(5, 0, 3)]: 1572.4090203619246
argmin_x[(5, 1, 3)]: 5072.661702893768


In [100]:
travel_times = {}

## coeff_input: const, slope, x-coordinate of transition point
# coeff_input = np.array([19.4, 0.1256, 0.786*1650]).reshape((3, 1)) @ np.ones((1, num_edges))

for e in range(num_edges):
    for t in range(T):
        travel_times[(e, 0, t)] = coeff_input[0, e] + coeff_input[1, e] * max(argmin_x[(e, 0, t)] - coeff_input[2, e], 0.0)
        travel_times[(e, 1, t)] = coeff_input[0, e] + coeff_input[1, e] * max(argmin_x[(e, 1, t)]/num_gp_lanes - coeff_input[2, e], 0.0)

edge_demand = {}
avg_travel_time = {}
percent_on_express = {}
obj_E = np.zeros(num_edges)
obj_I = np.zeros(num_edges)
obj_R = np.zeros(num_edges)
obj = np.zeros(num_edges)

for e in range(num_edges):
    avg_travel_time[e, 'el'] = 0.0
    avg_travel_time[e, 'in'] = 0.0
    avg_travel_time[e, 'ex'] = 0.0
    avg_travel_time[e, 'gp'] = 0.0
    
    percent_on_express[e, 'el'] = 0.0
    percent_on_express[e, 'in'] = 0.0
    percent_on_express[e, 'all'] = 0.0
    
    obj_E[e] = 0.0
    obj_I[e] = 0.0
    obj_R[e] = 0.0
    obj[e] = 0.0
    
    for t in range(T):
        edge_demand[e, t, 'el'] = sum(argmin_y_in_el_total[(e, k, t, 'el')] for k in range(2))
        edge_demand[e, t, 'in'] = sum(argmin_y_in_el_total[(e, k, t, 'in')] for k in range(2))
        
    percent_on_express[e, 'el'] += sum(argmin_y_in_el_total[(e, 0, t, 'el')] for t in range(T)) \
                                    / sum(edge_demand[e, t, 'el'] for t in range(T))
    percent_on_express[e, 'in'] += sum(argmin_y_in_el_total[(e, 0, t, 'in')] for t in range(T)) \
                                    / sum(edge_demand[e, t, 'in'] for t in range(T))
    percent_on_express[e, 'all'] += sum(argmin_y_in_el_total[(e, 0, t, 'el')] + argmin_y_in_el_total[(e, 0, t, 'in')] for t in range(T)) \
                                    / sum(edge_demand[e, t, 'el'] + edge_demand[e, t, 'in'] for t in range(T))
    
    avg_travel_time[e, 'el'] += sum(argmin_y_in_el_total[(e, k, t, 'el')] * travel_times[(e, k, t)] for k in range(2) for t in range(T)) \
                                    / sum(edge_demand[e, t, 'el'] for t in range(T))    
    avg_travel_time[e, 'in'] += sum(argmin_y_in_el_total[(e, k, t, 'in')] * travel_times[(e, k, t)] for k in range(2) for t in range(T)) \
                                    / sum(edge_demand[e, t, 'in'] for t in range(T))    
#     avg_travel_time[e, 'ex'] += sum( (argmin_y_in_el_total[(e, 0, t, 'el')] + argmin_y_in_el_total[(e, 0, t, 'in')]) * travel_times[(e, 0, t)] for t in range(T)) \
#                                     / sum( argmin_y_in_el_total[(e, 0, t, 'el')] + argmin_y_in_el_total[(e, 0, t, 'in')] for t in range(T))
#     avg_travel_time[e, 'gp'] += sum( (argmin_y_in_el_total[(e, 1, t, 'el')] + argmin_y_in_el_total[(e, 1, t, 'in')]) * travel_times[(e, 0, t)] for t in range(T)) \
#                                     / sum( argmin_y_in_el_total[(e, 1, t, 'el')] + argmin_y_in_el_total[(e, 1, t, 'in')] for t in range(T))
    avg_travel_time[e, 'ex'] += sum(travel_times[(e, 0, t)] for t in range(T)) / T 
    avg_travel_time[e, 'gp'] += sum(travel_times[(e, 1, t)] for t in range(T)) / T 
    
    obj_E[e] = sum( argmin_y[(od, g, e, 0, t)] * VoT_array[od, g, t] * travel_times[e, 0, t] \
                        for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) \
                    + sum( argmin_y[(od, g, e, 1, t)] * (VoT_array[od, g, t] * travel_times[e, 0, t] + argmin_tau[e, t]) \
                        for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) \
                    + sum( argmin_y[(od, g, e, 2, t)] * VoT_array[od, g, t] * travel_times[e, 1, t] \
                          for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) 

    obj_I[e] = sum( argmin_y[(od, g, e, 0, t)] * (VoT_array[od, g, t] * travel_times[e, 0, t] + argmin_tau[e, t]) \
                        for od in edge_to_od_dict[e] for g in in_indices for t in range(T) ) \
                    + sum( argmin_y[(od, g, e, 1, t)] * VoT_array[od, g, t] * travel_times[e, 1, t] \
                        for od in edge_to_od_dict[e] for g in in_indices for t in range(T) )

    obj_R[e] = sum( argmin_y[(od, g, e, 0, t)] * argmin_tau[e, t] \
                        for od in edge_to_od_dict[e] for g in in_indices for t in range(T) ) \
                    + sum( argmin_y[(od, g, e, 1, t)] * argmin_tau[e, t] \
                        for od in edge_to_od_dict[e] for g in el_indices for t in range(T) )

    obj[e] = lambda_E * obj_E[e] - lambda_R * obj_R[e] + lambda_I * obj_I[e]


# welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, argmin_tau, \
#                 demand_array, VoT_array, num_el, od_to_edges_array, y, \
#                 coeff_input)

# avg_travel_time
# percent_on_express
# obj_R



In [101]:
# e = 0
# od = 0
# g = 0
# t = 0

# # sum( argmin_y[(od, g, e, 0, t)] * VoT_array[od, g, t] * travel_times[e, 0, t] \
# #                         for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) 
# sum( argmin_y[(od, g, e, 1, t)] * (VoT_array[od, g, t] * travel_times[e, 0, t] + tau[e, t]) \
#     for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) 
# #                     + sum( argmin_y[(od, g, e, 2, t)] * VoT_array[od, g, t] * travel_times[e, 1, t] \
# #                           for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) ) 

# # argmin_y[(od, g, e, 1, t)]



# # sum(edge_demand[e, t, 'el'] for t in range(T) )

In [102]:
# travel_times
argmin_x

{(0, 0, 0): 1154.4569491230097,
 (0, 1, 0): 3693.646683368588,
 (0, 0, 1): 1152.3711102408265,
 (0, 1, 1): 3695.7325222507716,
 (0, 0, 2): 1153.6543697785416,
 (0, 1, 2): 3694.449262713056,
 (0, 0, 3): 1156.8924117369897,
 (0, 1, 3): 3691.211220754608,
 (0, 0, 4): 1212.048861331159,
 (0, 1, 4): 3636.0547711604386,
 (1, 0, 0): 1285.0455758474127,
 (1, 1, 0): 3889.1852713729604,
 (1, 0, 1): 1231.0875977437638,
 (1, 1, 1): 3943.1432494766086,
 (1, 0, 2): 1228.1131917153532,
 (1, 1, 2): 3946.1176555050183,
 (1, 0, 3): 1235.8211632685047,
 (1, 1, 3): 3938.409683951868,
 (1, 0, 4): 1293.5837646835398,
 (1, 1, 4): 3880.6470825368315,
 (2, 0, 0): 1698.4810510456987,
 (2, 1, 0): 5095.236879139102,
 (2, 0, 1): 1698.4810510456987,
 (2, 1, 1): 5095.236879139102,
 (2, 0, 2): 1600.5752036891956,
 (2, 1, 2): 5193.1427264956055,
 (2, 0, 3): 1611.036134651079,
 (2, 1, 3): 5182.681795533723,
 (2, 0, 4): 1698.4810510456987,
 (2, 1, 4): 5095.236879139102,
 (3, 0, 0): 1511.9162329942628,
 (3, 1, 0): 4535.7

In [103]:
# print("argmin_x[(3, 0, 0)]:", argmin_x[(3, 0, 0)])
# print("argmin_x[(3, 1, 0)]:", argmin_x[(3, 1, 0)])

In [115]:
opt_data_array = np.zeros((num_edges, 16))

# argmin_tau
opt_data_array[:, 0:5] = argmin_tau

# argmin_tau_avg
opt_data_array[:, 5] = np.linalg.norm(argmin_tau, axis=1)

# B (as array)
opt_data_array[:, 6] = argmin_B * np.eye(1, num_edges, 0)

# percent_on_express (overall)
# percent_on_express (eligible)
# percent_on_express (ineligible)
opt_data_array[:, 7] = np.array([percent_on_express[e, 'all'] for e in range(num_edges)]) * 100
opt_data_array[:, 8] = np.array([percent_on_express[e, 'el'] for e in range(num_edges)]) * 100
opt_data_array[:, 9] = np.array([percent_on_express[e, 'in'] for e in range(num_edges)]) * 100

# avg_travel_time (express lane)
# avg_travel_time (general purpose lane)
opt_data_array[:, 10] = np.array([avg_travel_time[e, 'ex'] for e in range(num_edges)])
opt_data_array[:, 11] = np.array([avg_travel_time[e, 'gp'] for e in range(num_edges)])

# obj_E = {}
# obj_I = {}
# obj_R = {}
# obj
opt_data_array[:, 12] = np.array([obj_E[e] for e in range(num_edges)]) 
opt_data_array[:, 13] = np.array([obj_I[e] for e in range(num_edges)]) 
opt_data_array[:, 14] = np.array([obj_R[e] for e in range(num_edges)]) 
opt_data_array[:, 15] = np.array([obj[e] for e in range(num_edges)]) 


opt_data_array = np.round(opt_data_array, decimals=2)

In [116]:
column_names = []
column_names += ["tau (t=" + str(t+1) + ")" for t in range(T) ]
column_names += ["tau (time-averaged)", \
                 "B", \
                 "% overall users using express lanes", \
                 "% eligible users using express lanes", \
                 "% ineligible users using express lanes", \
                 "Average travel time (express lanes)", \
                 "Average travel time (general purpose lanes)", \
                 "Total travel cost (eligible users)", \
                 "Total travel cost (ineligible users)", \
                 "Total toll revenue", \
                 "Total societal cost"]

row_names = ["e=" + str(k+1) for k in range(num_edges) ]

df_opt_save = pd.DataFrame(opt_data_array, index=row_names, columns=column_names)

df_opt_save



,tau (t=1),tau (t=2),tau (t=3),tau (t=4),tau (t=5),tau (time-averaged),B,% overall users using express lanes,% eligible users using express lanes,% ineligible users using express lanes,Average travel time (express lanes),Average travel time (general purpose lanes),Total travel cost (eligible users),Total travel cost (ineligible users),Total toll revenue,Total societal cost
e=1,0.11,0.11,0.11,0.11,0.00,0.21,0.0,24.05,0.0,27.42,1.56,1.61,277.57,40686.61,491.75,40472.42
e=2,0.01,0.08,0.08,0.08,0.00,0.14,0.0,24.25,0.0,27.79,2.36,2.39,481.84,59405.25,322.96,59564.13
e=3,0.00,0.00,0.33,0.33,0.00,0.47,0.0,24.46,0.0,27.93,6.11,6.19,1668.50,200521.50,1070.11,201119.89
e=4,0.00,0.34,0.00,0.07,0.34,0.49,0.0,24.48,0.0,27.91,1.60,1.69,406.62,48155.02,1088.96,47472.68
e=5,0.00,0.00,0.53,0.00,0.53,0.76,0.0,24.46,0.0,27.77,7.49,7.62,2385.20,280526.58,2008.68,280903.10
e=6,0.06,0.00,0.06,0.06,0.00,0.11,0.0,24.19,0.0,27.33,1.74,1.76,442.68,54746.86,302.23,54887.31
e=7,0.07,0.00,0.00,0.00,0.07,0.10,0.0,24.47,0.0,27.65,2.67,2.69,730.92,88974.42,244.63,89460.71


In [117]:
random_string = ""
for idx_rand_str in range(10):
    random_string += str(np.random.randint(1, 9))

directory_to_save = "../data/opt_tolls_subsidies_metrics/"
random_filename = "opt___" + random_string + '.csv'

df_opt_save.to_csv(directory_to_save + random_filename)

# Compute good initializations for alpha:

In [ ]:
# # el_indices
# # in_indices

# e = 0
# # VoT_array[edge_to_od_dict[e], in_indices, :]

# VoT_array_trunc_in_temp = VoT_array[edge_to_od_dict[e], :]
# VoT_array_trunc_in = VoT_array_trunc_in_temp[:, in_indices, :]

# # print("VoT_array[:, in_indices, :]:\n", VoT_array[:, in_indices, :])
# # print()
# # print("VoT_array[edge_to_od_dict[e], :, :]:\n", VoT_array[edge_to_od_dict[e], :, :])

# # VoT_array_trunc_in

In [ ]:
# demand_array.shape

In [108]:
arr_temp = np.stack([np.array([1, 2, 3]), np.array([3, 1, 4])]).T
arr_temp[0:, :]

array([[1, 3],
       [2, 1],
       [3, 4]])

In [110]:
alpha_init = np.zeros((num_edges, T))

for e in range(num_edges):
#     print("edge_to_od_dict[e]:", edge_to_od_dict[e])
    
    for t in range(T):
        
        ## Ineligible users:
        
        VoT_array_trunc_in_temp = VoT_array[edge_to_od_dict[e], :, t]
        VoT_array_trunc_in = VoT_array_trunc_in_temp[:, in_indices]
        demand_array_trunc_in_temp = demand_array[edge_to_od_dict[e], :]
        demand_array_trunc_in = demand_array_trunc_in_temp[:, in_indices]

        VoT_array_trunc_in_flattened = VoT_array_trunc_in.flatten()
        demand_array_trunc_in_flattened = demand_array_trunc_in.flatten()
        VoT_demand_in = np.stack([VoT_array_trunc_in_flattened, demand_array_trunc_in_flattened]).T    
        VoT_demand_in_sorted = VoT_demand_in[np.argsort(VoT_demand_in[:, 0])]
        VoT_demand_in_cumul = np.zeros(VoT_demand_in_sorted.shape)
        VoT_demand_in_cumul[:, 0] = VoT_demand_in_sorted[:, 0]
        for row_index in range(VoT_demand_in_cumul.shape[0]):
            VoT_demand_in_cumul[row_index, 1] = np.sum(VoT_demand_in_sorted[row_index:, 1])

        ## Eligible users:

        # Flatten VoT_array_trunc_in and demand_array_trunc_in, and concatenate.
        # Sort by VoT
        # Create cumulative demand array

        VoT_array_trunc_el_temp = VoT_array[edge_to_od_dict[e], :, t]
        VoT_array_trunc_el = VoT_array_trunc_el_temp[:, el_indices]
        demand_array_trunc_el_temp = demand_array[edge_to_od_dict[e], :]
        demand_array_trunc_el = demand_array_trunc_el_temp[:, el_indices]

        VoT_array_trunc_el_flattened = VoT_array_trunc_el.flatten()
        demand_array_trunc_el_flattened = demand_array_trunc_el.flatten()
        VoT_demand_el = np.stack([VoT_array_trunc_el_flattened, demand_array_trunc_el_flattened]).T    
        VoT_demand_el_sorted = VoT_demand_el[np.argsort(VoT_demand_el[:, 0])]
        VoT_demand_el_cumul = np.zeros(VoT_demand_el_sorted.shape)
        VoT_demand_el_cumul[:, 0] = VoT_demand_el_sorted[:, 0]
        for row_index in range(VoT_demand_el_cumul.shape[0]):
            VoT_demand_el_cumul[row_index, 1] = np.sum(VoT_demand_el_sorted[row_index:, 1])
    
#     VoT_array[od, g, t]

VoT_in_boundary = np.zeros((num_edges, T))
VoT_el_boundary = np.zeros((num_edges, T))

for e in range(num_edges):
    for t in range(T):
        
# for e in [0]:
#     for t in [0]:
        
        print()
        print("e:", e)
        print("t:", t)
        
        ## Ineligible users:
        
        # Find minimum entry in cumulative demand array such that demand >= argmin_y_in_el_total[(e, 0, t, 'in')]
        # Find corresponding VoT index -> Value
        
        VoT_in_boundary_index = max([row_index for row_index in range(VoT_demand_in_cumul.shape[0]) \
                                     if VoT_demand_in_cumul[row_index, 1] >= argmin_y_in_el_total[(e, 0, t, 'in')]])
        VoT_in_boundary[e, t] = VoT_demand_in_cumul[VoT_in_boundary_index, 0]
        
        ## Eligible users:
        
        # Find minimum entry in cumulative demand array such that demand >= argmin_y_in_el_total[(e, 0, t, 'in')]
        # Find corresponding VoT index -> Value
        
        VoT_el_boundary_index = max([row_index for row_index in range(VoT_demand_el_cumul.shape[0]) \
                                     if VoT_demand_el_cumul[row_index, 1] >= argmin_y_in_el_total[(e, 0, t, 'el')]])
        VoT_el_boundary[e, t] = VoT_demand_el_cumul[VoT_el_boundary_index, 0]
        
        if argmin_y_in_el_total[(e, 0, t, 'el')] <= 1E-3:
            alpha_init[e, t] = 0
        else:
            alpha_init[e, t] = 1.0 - VoT_el_boundary[e, t]/VoT_in_boundary[e, t]
        
        print("VoT_in_boundary_index:", VoT_in_boundary_index)
        print("VoT_el_boundary_index:", VoT_el_boundary_index)
        print("VoT_in_boundary[e, t]:", VoT_in_boundary[e, t])
        print("VoT_el_boundary[e, t]:", VoT_el_boundary[e, t])
        print("alpha_init[e, t]:", alpha_init[e, t])
        
        assert 0.0 <= alpha_init[e, t] <= 1.0



e: 0
t: 0
VoT_in_boundary_index: 19
VoT_el_boundary_index: 13
VoT_in_boundary[e, t]: 1.916291942971816
VoT_el_boundary[e, t]: 0.1081094831094831
alpha_init[e, t]: 0.0

e: 0
t: 1
VoT_in_boundary_index: 19
VoT_el_boundary_index: 13
VoT_in_boundary[e, t]: 1.916291942971816
VoT_el_boundary[e, t]: 0.1081094831094831
alpha_init[e, t]: 0.0

e: 0
t: 2
VoT_in_boundary_index: 19
VoT_el_boundary_index: 13
VoT_in_boundary[e, t]: 1.916291942971816
VoT_el_boundary[e, t]: 0.1081094831094831
alpha_init[e, t]: 0.0

e: 0
t: 3
VoT_in_boundary_index: 18
VoT_el_boundary_index: 13
VoT_in_boundary[e, t]: 1.850362482635435
VoT_el_boundary[e, t]: 0.1081094831094831
alpha_init[e, t]: 0.0

e: 0
t: 4
VoT_in_boundary_index: 18
VoT_el_boundary_index: 13
VoT_in_boundary[e, t]: 1.850362482635435
VoT_el_boundary[e, t]: 0.1081094831094831
alpha_init[e, t]: 0.0

e: 1
t: 0
VoT_in_boundary_index: 18
VoT_el_boundary_index: 13
VoT_in_boundary[e, t]: 1.850362482635435
VoT_el_boundary[e, t]: 0.1081094831094831
alpha_init[e, 

In [112]:
# # VoT_array_trunc_in_flattened
# # demand_in_trunc_in_flattened

# print("argmin_y_in_el_total[(e, 0, t, el)]:\n", argmin_y_in_el_total[(e, 0, t, 'el')])
# print()
# print("argmin_y_in_el_total[(e, 0, t, in)]:\n", argmin_y_in_el_total[(e, 0, t, 'in')])
# print()
# print("VoT_demand_el_cumul:\n", VoT_demand_el_cumul)
# print()
# print("VoT_demand_in_cumul:\n", VoT_demand_in_cumul)



In [113]:
# argmin_tau
# alpha_init

init_tau_alpha_array = np.block([argmin_tau, alpha_init])
init_tau_alpha_array

array([[0.1065    , 0.1065    , 0.1065    , 0.1065    , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.0118    , 0.0833    , 0.0833    , 0.0833    , 0.        ,
        0.94157389, 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.3332    , 0.3332    , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.3427    , 0.        , 0.0678    , 0.3427    ,
        0.        , 0.        , 0.        , 0.94157389, 0.        ],
       [0.        , 0.        , 0.5348    , 0.        , 0.5348    ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.0641    , 0.        , 0.0641    , 0.0641    , 0.        ,
        0.9392206 , 0.        , 0.9392206 , 0.9392206 , 0.        ],
       [0.0726    , 0.        , 0.        , 0.        , 0.0726    ,
        0.9392206 , 0.        , 0.        , 0.        , 0.9392206 ]])

In [114]:
column_names = []
column_names += ["tau (t=" + str(t+1) + ")" for t in range(T) ]
column_names += ["alpha (t=" + str(t+1) + ")" for t in range(T) ]

row_names = ["e=" + str(k+1) for k in range(num_edges) ]

df_inits_save = pd.DataFrame(init_tau_alpha_array, index=row_names, columns=column_names)

# df_inits_save

random_string = ""
for idx_rand_str in range(10):
    random_string += str(np.random.randint(1, 9))

directory_to_save = "../data/opt_tolls_subsidies_metrics/"
random_filename = "inits___" + random_string + '.csv'

df_inits_save.to_csv(directory_to_save + random_filename)

In [ ]:
# argmin_y_in_el_total[(e, 0, t, 'in')]

In [ ]:
# demand_array_trunc_in_flattened

# Testing solve CBCP direct with a small example:

In [ ]:
# od_to_edges_array = np.array([[0, 0], [0, 1], [1, 1], [1, 2]])
# od_to_edges_list_full = [[0], [0, 1], [1], [1, 2]]
# edge_to_ods = [[0, 1], [1, 2, 3], [3]]

# num_od = 3
# num_edges = 3
# num_gp_lanes = 3
# num_groups = 5
# T = 5
# el_indices = [0, 1]
# in_indices = [2, 3, 4]
# B = 3
# tau = np.zeros((num_edges, T))

# for e in range(num_edges):
#     for t in range(T):
#         tau[e, t] = 1.5 + 0.5 * e + 0.2 * t

# demand_array = np.zeros((num_od, num_groups))
# VoT_array = np.zeros((num_od, num_groups, T))

# for od in range(num_od):
#     for g in range(num_groups):
#         demand_array[od, g] = 1.0 + od + 0.1 * g
#         VoT_array[od, g, t] = 2.0 + od + 0.1 * g
        
# coeff_input = np.array([1, 1, 0, 0, 0])
# latency_params_length = coeff_input.shape[0]

# ex_to_gp_multiplier = np.array([1/num_gp_lanes**p for p in range(latency_params_length)]).reshape((latency_params_length, 1)) \
#                         @ np.ones((1, num_edges))
# a = np.zeros((latency_params_length, num_edges, 2))
# coeff[:, :, 0] = coeff_input.reshape((latency_params_length, 1)) @ np.ones((1, num_edges))
# coeff[:, :, 1] = coeff[:, :, 0] * ex_to_gp_multiplier


# y_vals = solve_CBCP_direct(T, num_edges, num_gp_lanes, tau, B, od_to_edges_array, \
#                            demand_array, VoT_array, num_el, coeff_input)


# # y = {}
# # for od in range(num_od):
# #     for e in od_to_edges_list_full[od]:
# #         for t in range(T):
# #             for g in el_indices:
# #                 for k in [0, 1, 2]:
# #                     y[(od, g, e, k, t)] = cp.Variable(1)
# #             for g in in_indices:
# #                 for k in [0, 1]:
# #                     y[(od, g, e, k, t)] = cp.Variable(1)

# # x = {}
# # for e in range(num_edges):
# #     for k in [0, 1]:
# #         for t in range(T):
# #             x[(e, k, t)] = cp.Variable(1)

# # # Objective:
# # func = 0.0
# # for e in range(num_edges):

# # for od in range(num_od):
# #     for e in od_to_edges_list_full[od]:
# #         for t in range(T):
# #             for g in el_indices:
# #                 func += tau[e, t] * y[(od, g, e, 1, t)] / VoT_array[od, g, t]
# #             for g in in_indices:
# #                 func += tau[e, t] * y[(od, g, e, 0, t)] / VoT_array[od, g, t]

# # objective = cp.Minimize(func)

# # # Constraints:
# # constraints = []

# # constraints += [y[(od, g, e, k, t)] >= 0.0 for od in range(num_od) \
# #                 for e in od_to_edges_list_full[od] for g in el_indices  \
# #                 for k in [0, 1, 2] for t in range(T)]
# # constraints += [y[(od, g, e, k, t)] >= 0.0 for od in range(num_od) \
# #                 for e in od_to_edges_list_full[od] for g in in_indices  \
# #                 for k in [0, 1] for t in range(T)]


# # for e in range(num_edges):
# #     for t in range(T):
# # #         print("e:", e)
# # #         print("edge_to_ods[e]:", edge_to_ods[e])

# #         ## Edge contraints:
# #         constraints += [sum( y[(od, g, e, 0, t)] + y[(od, g, e, 1, t)] for od in edge_to_ods[e] for g in el_indices) \
# #                             + sum( y[(od, g, e, 0, t)] for od in edge_to_ods[e] for g in in_indices ) \
# #                             == x[(e, 0, t)] ]
# #         constraints += [sum( y[(od, g, e, 2, t)] for od in edge_to_ods[e] for g in el_indices) \
# #                             + sum( y[(od, g, e, 1, t)] for od in edge_to_ods[e] for g in in_indices ) \
# #                             == x[(e, 1, t)] ]

# #         ## Group flow constraints:
# #         constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1, 2]) == demand_array[od, g] \
# #                         for od in edge_to_ods[e] for g in el_indices]
# #         constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1]) == demand_array[od, g] \
# #                         for od in edge_to_ods[e] for g in in_indices]

# # constraints += [sum(y[(od, g, e, 0, t)] * tau[e, t] for e in od_to_edges_list_full[od] for t in range(T)) \
# #                 <= B * demand_array[od, g] for od in range(num_od) for g in el_indices]

# # # Problem:
# # prob = cp.Problem(objective, constraints)

# # # Solve:
# # result = prob.solve()


# # assert prob.status != "infeasible", "problem.status should not be infeasible."
# # assert prob.status != "unbounded", "problem.status should not be unbounded."
# # print()
# # print("prob.status:", prob.status)

# # # Extract Values:
# # y_values = {}
# # for e in range(num_edges):
# #     for od in edge_to_ods[e]:
# #         for t in range(T):
# #             for g in el_indices:
# #                 for k in [0, 1, 2]:
# #                     print("y[(od, g, e, k, t)].value:", y[(od, g, e, k, t)].value)
# #                     y_values[(od, g, e, k, t)] = max(y[(od, g, e, k, t)].value[0], 0.0)
# #             for g in in_indices:
# #                 for k in [0, 1]:
# #                     print("y[(od, g, e, k, t)].value:", y[(od, g, e, k, t)].value)
# #                     y_values[(od, g, e, k, t)] = max(y[(od, g, e, k, t)].value[0], 0.0)




In [ ]:
# demand_edges_array

In [ ]:
# np.sum(demand_array[3:16, :])/5

In [ ]:
print("first(welfare_list):", welfare_list[0])
print("min(welfare_list):", min(welfare_list))
print("max(welfare_list):", max(welfare_list))
print("argmin_tau:", argmin_tau)
print("argmin_B:", argmin_B)

## Test:

## <font color='red'>Colored Font Titles</font> 

# Scratch Work:

In [ ]:
x = cp.Variable(2)
y = cp.Variable(2)
v_fixed = np.array([0, 1])
objective = cp.Minimize(cp.sum_squares(x - y) + cp.sum_squares(x - v_fixed))
constraints = []
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
print("x.value:", x.value)
print("y.value:", y.value)
print()


## CVXPY can handle 4d arrays:

In [ ]:

I, J, K, L = 2, 3, 4, 5

# Variables:
x_test = {}
for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                x_test[(i, j, k, ell)] = cp.Variable(1)
            
# Objective:
func = 0.0
func += cp.sum([x_test[(i, j, k, ell)]**2 for i in range(I) for j in range(J) \
                for k in range(K) for ell in range(L)])
            
objective = cp.Minimize(func)

# Constraints:
constraints = []

for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                constraints += [cp.sum([x_test[(i, j, k, ell)] for i in range(I) for j in range(J) \
                                        for k in range(K) for ell in range(L) ]) == 1.0]
                constraints += [x_test[(i, j, k, ell)] >= 0.0 for i in range(I) for j in range(J) \
                                        for k in range(K) for ell in range(L)]

# Solve problem:
prob = cp.Problem(objective, constraints)
result = prob.solve()

# Print solution:
for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                print("i, j, k, ell:", i, j, k, ell)
                print("x_test[(i,j,k, ell)].value:", x_test[(i, j, k, ell)].value)


## Linear Approximation for Latency Function:

In [ ]:
# Variables:
v = cp.Variable(1)
            
# Objective:
func = v - 1 + cp.square(cp.maximum(v-1, 0))
objective = cp.Minimize(func)

# Constraints:
constraints = [-3.0 <= v, v <= 3.0]

# Solve problem:
prob = cp.Problem(objective, constraints)
result = prob.solve()

# Print solution:
print("v.value:", v.value)
